# Project-5 Etude de Marché-Part-1

L'objectif est d'aider à cibler certains pays, en approfondissant l'étude de marché. 

Plus précisément, l'idéal serait de produire des "groupes" de pays par différentes méthodes.

La stratégie consiste à exporter les produits plutôt que de les produire localement, c'est-à-dire dans les nouveaux pays ciblés.

# Mission

Identifier les pays susceptibles d'être inclus dans le marché du poulet. 

L'échantillon contient tous les pays disponibles, chacun étant caractérisé par ces variables :

1. la différence de population entre une année précédente (votre choix) et l'année en cours, exprimée en pourcentage ;

2. proportion de protéines animales par rapport aux protéines totales dans la disponibilité alimentaire du pays ;
disponibilité alimentaire en protéines par habitant ;

3. disponibilité alimentaire en calories par habitant.

4. un dendrogramme contenant l'ensemble des pays étudiés, puis découpez-le pour obtenir 5 groupes.

5. le calcul de la position du centroïde de chaque groupe. 

6. courte liste de pays à cibler, présentant leurs caractéristiques. A

7. Visualisez vos partitions dans le premier plan factoriel obtenu par l'ACP.

8. Ensuite, effectuez les tests statistiques suivants :

un test d'adéquation : parmi les 4 variables, ou parmi d'autres variables que vous trouvez pertinentes, trouvez une variable dont la distribution est normale ;
un test de comparaison de deux populations (dans le cas gaussien) : choisissez 2 clusters parmi ceux que vous avez déterminés. Sur ces 2 clusters, testez la variable gaussienne par un test de comparaison.


In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

# Data Importation

In [2]:
population=pd.read_csv("pop_2008_2018.csv")
vegetal_data=pd.read_csv("vegetal_2017.csv")
animal_data=pd.read_csv("animal_2017.csv")
pib_habitant=pd.read_csv("pib_habitant.csv")
poulet_export_value=pd.read_csv("poulet_export_value.csv")
poluet_import_value=pd.read_csv("poulet_import_value.csv")

# Nettoyage des données

# 1. Population_data

In [3]:
population.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2008,2008,1000 personnes,27722.276,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2018,2018,1000 personnes,37171.921,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,202,Afrique du Sud,511,Population totale,3010,Population-Estimations,2008,2008,1000 personnes,49779.471,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,202,Afrique du Sud,511,Population totale,3010,Population-Estimations,2018,2018,1000 personnes,57792.518,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,3,Albanie,511,Population totale,3010,Population-Estimations,2008,2008,1000 personnes,3002.678,X,Sources internationales sûres,NaN


In [4]:
col_mult = ["Valeur"]
for i in col_mult:
    population[i] *= 1000

population.head(5)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2008,2008,1000 personnes,27722276.0,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2018,2018,1000 personnes,37171921.0,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,202,Afrique du Sud,511,Population totale,3010,Population-Estimations,2008,2008,1000 personnes,49779471.0,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,202,Afrique du Sud,511,Population totale,3010,Population-Estimations,2018,2018,1000 personnes,57792518.0,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,3,Albanie,511,Population totale,3010,Population-Estimations,2008,2008,1000 personnes,3002678.0,X,Sources internationales sûres,NaN


In [5]:
population_total = population.pivot_table(index=["Zone","Code zone"], columns = ["Année"], values=["Valeur"])

In [6]:
population_total = population_total.reset_index()

In [7]:
# calcul du ratio d'evolution entre les deux annees en %
population_total["ratio_evol_pop"]=round(((population_total[("Valeur", 2018)] - population_total[("Valeur", 2008)]) / population_total[("Valeur", 2008)]) * 100,1)


In [8]:
# mise en place des colonnes
population_total.columns = ["Zone", "Code zone", "2008", "2018", "ratio_evol_pop"]

In [9]:
population_total = population_total.reset_index()

In [10]:
population_total.head(5)

,index,Zone,Code zone,2008,2018,ratio_evol_pop
0,0,Afghanistan,2,27722276.0,37171921.0,34.1
1,1,Afrique du Sud,202,49779471.0,57792518.0,16.1
2,2,Albanie,3,3002678.0,2882740.0,-4.0
3,3,Algérie,4,34730608.0,42228408.0,21.6
4,4,Allemagne,79,81065752.0,83124418.0,2.5


In [11]:
population_total.shape

(237, 6)

In [12]:
population_total.isna().sum()

index             0
Zone              0
Code zone         0
2008              7
2018              1
ratio_evol_pop    8
dtype: int64

In [13]:
population_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 6 columns):
index             237 non-null int64
Zone              237 non-null object
Code zone         237 non-null int64
2008              230 non-null float64
2018              236 non-null float64
ratio_evol_pop    229 non-null float64
dtypes: float64(3), int64(2), object(1)
memory usage: 11.2+ KB


# 2.  Animal_data

In [14]:
animal_data.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2731,Viande de Bovins,2017,2017,Milliers de tonnes,128.0,Im,Données de la FAO basées sur une méthodologie ...
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2731,Viande de Bovins,2017,2017,Milliers de tonnes,6.0,Im,Données de la FAO basées sur une méthodologie ...
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2731,Viande de Bovins,2017,2017,Milliers de tonnes,-1.0,Im,Données de la FAO basées sur une méthodologie ...
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2731,Viande de Bovins,2017,2017,Milliers de tonnes,0.0,Im,Données de la FAO basées sur une méthodologie ...
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2731,Viande de Bovins,2017,2017,Milliers de tonnes,136.0,S,Données standardisées


In [15]:
animal_data.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2731,Viande de Bovins,2017,2017,Milliers de tonnes,128.0,Im,Données de la FAO basées sur une méthodologie ...
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2731,Viande de Bovins,2017,2017,Milliers de tonnes,6.0,Im,Données de la FAO basées sur une méthodologie ...
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2731,Viande de Bovins,2017,2017,Milliers de tonnes,-1.0,Im,Données de la FAO basées sur une méthodologie ...
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2731,Viande de Bovins,2017,2017,Milliers de tonnes,0.0,Im,Données de la FAO basées sur une méthodologie ...
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2731,Viande de Bovins,2017,2017,Milliers de tonnes,136.0,S,Données standardisées


In [16]:
animal_data.Valeur.sum()

4954219.82

# 3. Vegatal_data

In [17]:
vegetal_data.head(2)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,Im,Données de la FAO basées sur une méthodologie ...
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,3080.0,Im,Données de la FAO basées sur une méthodologie ...


#### Add the variables in animal and vegetal data

In [18]:
animal_data["origin"] = "Animal"
vegetal_data["origin"] = "Vegetal"
# we regroup the animal and vegetal data in different dataframe
ani_veg = animal_data.append(vegetal_data)

In [19]:
del animal_data, vegetal_data

In [20]:
# delete animal and vegetal
#del animal, vegetal

#Pivot table 
data_ani_veg = ani_veg.pivot_table(index=["Zone","Produit","origin", "Année"], columns = ["Élément"], values=["Valeur"], aggfunc=sum)

data_ani_veg.columns = ['Production', 'Importations - Quantité', 'Variation de stock',
       'Exportations - Quantité', 'Disponibilité intérieure', 'Aliments pour animaux', 'Semences',
       'Pertes', 'Traitement', 'Autres utilisations (non alimentaire)', 'Nourriture', 'Alimentation pour touristes', 'Résidus', 
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)']

data_ani_veg = data_ani_veg.reset_index()


col_mult = ["Production", "Importations - Quantité", "Variation de stock", "Exportations - Quantité", "Disponibilité intérieure", "Aliments pour animaux", "Alimentation pour touristes", "Traitement", "Nourriture", "Semences", "Pertes", "Résidus", "Autres utilisations (non alimentaire)"]

for i in col_mult:
    data_ani_veg[i] *= 1000000
    
data_ani_veg.head(5)

,Zone,Produit,origin,Année,Production,Importations - Quantité,Variation de stock,Exportations - Quantité,Disponibilité intérieure,Aliments pour animaux,...,Pertes,Traitement,Autres utilisations (non alimentaire),Nourriture,Alimentation pour touristes,Résidus,Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité de matière grasse en quantité (g/personne/jour)
0,Afghanistan,Abats Comestible,Animal,2017,NaN,NaN,NaN,5000000.0,1600000.0,190000.0,...,61000000.0,NaN,6000000.0,58000000.0,3000000.0,55000000.0,0.0,NaN,0.0,NaN
1,Afghanistan,"Agrumes, Autres",Vegetal,2017,NaN,NaN,NaN,1000000.0,1360000.0,10000.0,...,51000000.0,0.0,33000000.0,49000000.0,2000000.0,18000000.0,0.0,NaN,0.0,0.0
2,Afghanistan,"Alcool, non Comestible",Vegetal,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN
3,Afghanistan,Aliments pour enfants,Vegetal,2017,NaN,NaN,NaN,1000000.0,60000.0,0.0,...,2000000.0,NaN,4000000.0,2000000.0,NaN,NaN,0.0,NaN,NaN,1.0
4,Afghanistan,Ananas et produits,Vegetal,2017,NaN,NaN,NaN,0.0,20000.0,0.0,...,1000000.0,0.0,1000000.0,1000000.0,NaN,0.0,0.0,NaN,0.0,0.0


In [21]:
data_ani_veg.shape

(16314, 21)

In [22]:
#Merge the datasets
population_data = population_total[["Zone","Code zone","2008", "2018", "ratio_evol_pop"]]

# Merge data & population_
population_bilan = pd.merge(data_ani_veg,population_data,left_on= "Zone", right_on= "Zone")

population_bilan.head(5)

,Zone,Produit,origin,Année,Production,Importations - Quantité,Variation de stock,Exportations - Quantité,Disponibilité intérieure,Aliments pour animaux,...,Alimentation pour touristes,Résidus,Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité de matière grasse en quantité (g/personne/jour),Code zone,2008,2018,ratio_evol_pop
0,Afghanistan,Abats Comestible,Animal,2017,NaN,NaN,NaN,5000000.0,1600000.0,190000.0,...,3000000.0,55000000.0,0.0,NaN,0.0,NaN,2,27722276.0,37171921.0,34.1
1,Afghanistan,"Agrumes, Autres",Vegetal,2017,NaN,NaN,NaN,1000000.0,1360000.0,10000.0,...,2000000.0,18000000.0,0.0,NaN,0.0,0.0,2,27722276.0,37171921.0,34.1
2,Afghanistan,"Alcool, non Comestible",Vegetal,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,2,27722276.0,37171921.0,34.1
3,Afghanistan,Aliments pour enfants,Vegetal,2017,NaN,NaN,NaN,1000000.0,60000.0,0.0,...,NaN,NaN,0.0,NaN,NaN,1.0,2,27722276.0,37171921.0,34.1
4,Afghanistan,Ananas et produits,Vegetal,2017,NaN,NaN,NaN,0.0,20000.0,0.0,...,NaN,0.0,0.0,NaN,0.0,0.0,2,27722276.0,37171921.0,34.1


In [23]:
population_bilan.shape

(16258, 25)

In [24]:
print(pd.isnull(population_bilan).sum())

Zone                                                                 0
Produit                                                              0
origin                                                               0
Année                                                                0
Production                                                       13459
Importations - Quantité                                          12303
Variation de stock                                               10781
Exportations - Quantité                                           1623
Disponibilité intérieure                                          1505
Aliments pour animaux                                             1602
Semences                                                          1602
Pertes                                                               0
Traitement                                                        3823
Autres utilisations (non alimentaire)                              739
Nourri

In [25]:
#Replace the null values
population_bilan = population_bilan.fillna(population_bilan.mean())
population_bilan.head()

,Zone,Produit,origin,Année,Production,Importations - Quantité,Variation de stock,Exportations - Quantité,Disponibilité intérieure,Aliments pour animaux,...,Alimentation pour touristes,Résidus,Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité de matière grasse en quantité (g/personne/jour),Code zone,2008,2018,ratio_evol_pop
0,Afghanistan,Abats Comestible,Animal,2017,1.262951e+06,3.530108e+08,1.653094e+08,5.000000e+06,1.600000e+06,1.900000e+05,...,3.000000e+06,5.500000e+07,0.0,82.804022,0.000000,15.005179,2,27722276.0,37171921.0,34.1
1,Afghanistan,"Agrumes, Autres",Vegetal,2017,1.262951e+06,3.530108e+08,1.653094e+08,1.000000e+06,1.360000e+06,1.000000e+04,...,2.000000e+06,1.800000e+07,0.0,82.804022,0.000000,0.000000,2,27722276.0,37171921.0,34.1
2,Afghanistan,"Alcool, non Comestible",Vegetal,2017,1.262951e+06,3.530108e+08,1.653094e+08,3.384838e+07,7.979486e+06,1.043172e+06,...,1.041474e+08,0.000000e+00,0.0,82.804022,535.772338,15.005179,2,27722276.0,37171921.0,34.1
3,Afghanistan,Aliments pour enfants,Vegetal,2017,1.262951e+06,3.530108e+08,1.653094e+08,1.000000e+06,6.000000e+04,0.000000e+00,...,1.041474e+08,1.025647e+09,0.0,82.804022,535.772338,1.000000,2,27722276.0,37171921.0,34.1
4,Afghanistan,Ananas et produits,Vegetal,2017,1.262951e+06,3.530108e+08,1.653094e+08,0.000000e+00,2.000000e+04,0.000000e+00,...,1.041474e+08,0.000000e+00,0.0,82.804022,0.000000,0.000000,2,27722276.0,37171921.0,34.1


In [26]:
# Calcul de la disponibilité alimentaire
population_bilan["dispo_alim"] = population_bilan["Aliments pour animaux"]+population_bilan["Nourriture"]

In [27]:
# Regroupement de bilan par zones et origines
population_bilan_groupby = population_bilan.groupby(["Zone", "origin"]).sum()
population_bilan_groupby = population_bilan.reset_index()

In [28]:
population_bilan_groupby

,index,Zone,Produit,origin,Année,Production,Importations - Quantité,Variation de stock,Exportations - Quantité,Disponibilité intérieure,...,Résidus,Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité de matière grasse en quantité (g/personne/jour),Code zone,2008,2018,ratio_evol_pop,dispo_alim
0,0,Afghanistan,Abats Comestible,Animal,2017,1.262951e+06,3.530108e+08,1.653094e+08,5.000000e+06,1.600000e+06,...,5.500000e+07,0.0,82.804022,0.000000,15.005179,2,27722276.0,37171921.0,34.1,5.819000e+07
1,1,Afghanistan,"Agrumes, Autres",Vegetal,2017,1.262951e+06,3.530108e+08,1.653094e+08,1.000000e+06,1.360000e+06,...,1.800000e+07,0.0,82.804022,0.000000,0.000000,2,27722276.0,37171921.0,34.1,4.901000e+07
2,2,Afghanistan,"Alcool, non Comestible",Vegetal,2017,1.262951e+06,3.530108e+08,1.653094e+08,3.384838e+07,7.979486e+06,...,0.000000e+00,0.0,82.804022,535.772338,15.005179,2,27722276.0,37171921.0,34.1,3.522752e+08
3,3,Afghanistan,Aliments pour enfants,Vegetal,2017,1.262951e+06,3.530108e+08,1.653094e+08,1.000000e+06,6.000000e+04,...,1.025647e+09,0.0,82.804022,535.772338,1.000000,2,27722276.0,37171921.0,34.1,2.000000e+06
4,4,Afghanistan,Ananas et produits,Vegetal,2017,1.262951e+06,3.530108e+08,1.653094e+08,0.000000e+00,2.000000e+04,...,0.000000e+00,0.0,82.804022,0.000000,0.000000,2,27722276.0,37171921.0,34.1,1.000000e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16253,16253,Îles Salomon,Viande de Volailles,Animal,2017,0.000000e+00,3.530108e+08,1.653094e+08,1.900000e+07,4.560000e+06,...,0.000000e+00,0.0,82.804022,535.772338,2.000000,25,503360.0,652857.0,29.7,4.390000e+06
16254,16254,Îles Salomon,Viande de porcins,Animal,2017,0.000000e+00,3.530108e+08,1.653094e+08,4.300000e+07,4.390000e+06,...,2.000000e+06,0.0,82.804022,535.772338,0.000000,25,503360.0,652857.0,29.7,7.140000e+06
16255,16255,Îles Salomon,"Viande, Autre",Animal,2017,0.000000e+00,3.530108e+08,1.653094e+08,0.000000e+00,1.000000e+04,...,1.025647e+09,0.0,82.804022,535.772338,0.000000,25,503360.0,652857.0,29.7,0.000000e+00
16256,16256,Îles Salomon,Vin,Vegetal,2017,0.000000e+00,3.530108e+08,1.653094e+08,0.000000e+00,2.500000e+05,...,1.025647e+09,0.0,82.804022,535.772338,0.000000,25,503360.0,652857.0,29.7,0.000000e+00


#### Pivot table disponibilité alimentaire en proteine

In [29]:
protein_bilan = population_bilan_groupby.pivot_table(index=["Zone"],columns = ["origin"], values=["Disponibilité de protéines en quantité (g/personne/jour)"])

protein_bilan.columns = ["Disponibilité de protéines en quantité animal", "Disponibilité de protéines en quantité vegetal"]

protein_bilan = protein_bilan.reset_index()

# Ratio of protien animal
protein_bilan["ratio_protein_anim"] = round((protein_bilan["Disponibilité de protéines en quantité animal"] / (protein_bilan["Disponibilité de protéines en quantité vegetal"] + protein_bilan["Disponibilité de protéines en quantité animal"])) * 100,1)

# Show 30 countries with high ratio
protein_bilan.sort_values(["ratio_protein_anim"], axis=0, ascending=False, inplace=True)
protein_bilan = protein_bilan.reset_index()
protein_bilan.head(30)[["Zone","ratio_protein_anim"]]

,Zone,ratio_protein_anim
0,Yémen,79.3
1,Rwanda,75.9
2,Congo,75.8
3,Niger,75.7
4,République populaire démocratique de Corée,75.7
5,Tchad,74.6
6,Sénégal,74.0
7,Guinée,73.6
8,Cambodge,73.3
9,République centrafricaine,72.8


## 3.  Disponibilité alimentaire en protéines par habitant & disponibilité alimentaire en calories par habitant.¶

In [30]:
dispo_alimentaire = protein_bilan.groupby(["Zone"]).sum()
dispo_alimentaire.head()

,index,Disponibilité de protéines en quantité animal,Disponibilité de protéines en quantité vegetal,ratio_protein_anim
Zone,,,,
Afghanistan,0,239.424113,101.264170,70.3
Afrique du Sud,1,527.328141,605.544592,46.5
Albanie,2,515.691777,338.928327,60.3
Algérie,3,515.545084,357.439360,59.1
Allemagne,4,1048.425337,973.817341,51.8


In [31]:
dispo_alimentaire = population_bilan.groupby(["Zone"]).sum()
dispo_alimentaire.head()

,Année,Production,Importations - Quantité,Variation de stock,Exportations - Quantité,Disponibilité intérieure,Aliments pour animaux,Semences,Pertes,Traitement,...,Résidus,Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité de matière grasse en quantité (g/personne/jour),Code zone,2008,2018,ratio_evol_pop,dispo_alim
Zone,,,,,,,,,,,,,,,,,,,,,
Afghanistan,175479,1.098767e+08,2.439672e+10,1.384068e+10,2.555726e+09,4.417343e+08,4.594441e+07,6.967323e+07,1.616500e+10,4.501870e+09,...,2.182447e+10,68.521431,6637.301682,10467.902090,1377.139832,174,2.411838e+09,3.233957e+09,2966.7,1.676050e+10
Afrique du Sud,191615,1.199804e+08,3.005494e+10,1.218971e+10,3.204787e+09,6.048259e+08,9.087538e+07,9.133041e+07,6.183900e+10,1.142787e+10,...,8.418941e+10,-606.264264,6503.105705,55805.974328,3412.061790,19190,4.729050e+09,5.490289e+09,1529.5,3.373983e+10
Albanie,185564,1.161915e+08,2.644695e+10,9.914254e+09,3.862877e+09,1.289003e+09,1.328544e+08,1.316132e+08,4.885070e+09,3.787963e+09,...,2.987688e+10,-17.264264,6680.321794,35070.201990,184.077685,276,2.762464e+08,2.652121e+08,-368.0,8.445003e+09
Algérie,183547,1.149285e+08,3.161382e+10,1.251015e+10,3.576939e+09,8.142964e+08,8.746220e+07,9.955661e+07,4.340471e+10,3.745395e+09,...,4.999807e+10,366.735736,6635.105705,35847.201990,1558.082864,364,3.160485e+09,3.842785e+09,1965.6,3.393775e+10
Allemagne,191615,1.199804e+08,5.097289e+10,1.510418e+10,3.815787e+09,9.053959e+08,1.543054e+08,1.115304e+08,1.752650e+11,5.198962e+10,...,1.946269e+11,-667.264264,8432.929839,94154.023284,3910.072148,7505,7.701246e+09,7.896820e+09,237.5,7.255449e+10


In [32]:
dispo_alimentaire.columns = ['Code zone', 'Année', 'Production',
       'Importations - Quantité', 'Variation de stock',
       'Exportations - Quantité', 'Disponibilité intérieure',
       'Aliments pour animaux', 'Semences', 'Pertes', 'Traitement',
       'Autres utilisations (non alimentaire)', 'Nourriture',
       'Alimentation pour touristes', 'Résidus',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)',
       '2008', '2018', 'ratio_evol', 'dispo_alim']

dispo_alimentaire = dispo_alimentaire.reset_index()

In [33]:
dispo_alimentaire = dispo_alimentaire[["Zone","Disponibilité alimentaire (Kcal/personne/jour)","Disponibilité de protéines en quantité (g/personne/jour)"]]
dispo_alimentaire.head(5)

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour)
0,Afghanistan,10467.902090,1377.139832
1,Afrique du Sud,55805.974328,3412.061790
2,Albanie,35070.201990,184.077685
3,Algérie,35847.201990,1558.082864
4,Allemagne,94154.023284,3910.072148


In [34]:
population_data = population_total[["Zone","Code zone","ratio_evol_pop"]]

# Merge population_ & food
data_fin_1 = pd.merge(dispo_alimentaire,population_data,left_on="Zone", right_on="Zone")

data_fin_1.head(5)

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Code zone,ratio_evol_pop
0,Afghanistan,10467.902090,1377.139832,2,34.1
1,Afrique du Sud,55805.974328,3412.061790,202,16.1
2,Albanie,35070.201990,184.077685,3,-4.0
3,Algérie,35847.201990,1558.082864,4,21.6
4,Allemagne,94154.023284,3910.072148,79,2.5


In [35]:
# Now merge protein data 
protein_bilan=protein_bilan[["Zone","ratio_protein_anim"]]

data_fin_1 = pd.merge(data_fin_1,protein_bilan,left_on="Zone", right_on="Zone")

data_fin_1.set_index("Zone",inplace=True)

data_fin_1.head(5)

,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Code zone,ratio_evol_pop,ratio_protein_anim
Zone,,,,,
Afghanistan,10467.902090,1377.139832,2,34.1,70.3
Afrique du Sud,55805.974328,3412.061790,202,16.1,46.5
Albanie,35070.201990,184.077685,3,-4.0,60.3
Algérie,35847.201990,1558.082864,4,21.6,59.1
Allemagne,94154.023284,3910.072148,79,2.5,51.8


In [36]:
# look for null values
print(pd.isnull(data_fin_1).sum())

Disponibilité alimentaire (Kcal/personne/jour)              0
Disponibilité de protéines en quantité (g/personne/jour)    0
Code zone                                                   0
ratio_evol_pop                                              2
ratio_protein_anim                                          4
dtype: int64


In [37]:
data_fin_1 = data_fin_1.fillna(data_fin_1.mean())

In [38]:
# look for null values
print(pd.isnull(data_fin_1).sum())

Disponibilité alimentaire (Kcal/personne/jour)              0
Disponibilité de protéines en quantité (g/personne/jour)    0
Code zone                                                   0
ratio_evol_pop                                              0
ratio_protein_anim                                          0
dtype: int64


### Copy the full datasets

In [39]:
# Copy des données complètes 
data_fin_2 = data_fin_1.copy()
data_fin_2.columns = ["Disponibilité alimentaire (Kcal/personne/jour)","Disponibilité de protéines en quantité (g/personne/jour)", "Code zone","ratio_evol_pop","ratio_protein_anim"]

In [40]:
data_fin_2.shape

(177, 5)

# 4. PIB_PAR DATA

In [41]:
pib_habitant.head(2)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,MK,Indicateurs macro,2,Afghanistan,6119,Valeur (US $),22014,Produit intérieur brut par habitant,2017,2017,NaN,605.557362,Fc,Donnée calculée,NaN
1,MK,Indicateurs macro,2,Afghanistan,6119,Valeur (US $),22014,Produit intérieur brut par habitant,2018,2018,NaN,563.983834,Fc,Donnée calculée,NaN


In [42]:
col_mult = ["Valeur"]
for i in col_mult:
    pib_habitant[i] *= 1000000

pib_habitant.head(2)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,MK,Indicateurs macro,2,Afghanistan,6119,Valeur (US $),22014,Produit intérieur brut par habitant,2017,2017,NaN,605557362.0,Fc,Donnée calculée,NaN
1,MK,Indicateurs macro,2,Afghanistan,6119,Valeur (US $),22014,Produit intérieur brut par habitant,2018,2018,NaN,563983834.0,Fc,Donnée calculée,NaN


In [43]:
pib_par_habitant = pib_habitant.pivot_table(index=["Zone", "Code zone"], columns=["Année"], values=["Valeur"]).reset_index()
pib_par_habitant.head()

Zone Code zone        Valeur              
Année                                    2017          2018
0         Afghanistan         2  6.055574e+08  5.639838e+08
1      Afrique du Sud       202  6.153459e+09  6.412963e+09
2             Albanie         3  4.445132e+09  5.131909e+09
3             Algérie         4  4.051244e+09  4.136290e+09
4           Allemagne        79  4.464274e+10  4.799347e+10

In [44]:
pib_par_habitant.columns = ["Zone", "Code zone", "Pib_par_habitant_2017", "pib_par_habitant_2018"]

In [45]:
pib_par_habitant = pib_par_habitant.reset_index()
pib_par_habitant.head(2)

,index,Zone,Code zone,Pib_par_habitant_2017,pib_par_habitant_2018
0,0,Afghanistan,2,6.055574e+08,5.639838e+08
1,1,Afrique du Sud,202,6.153459e+09,6.412963e+09


## Full_data_Merge

In [46]:
data_full = pd.merge(data_fin_2, pib_par_habitant, left_on="Zone", right_on="Zone")

In [47]:
data_full

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Code zone_x,ratio_evol_pop,ratio_protein_anim,index,Code zone_y,Pib_par_habitant_2017,pib_par_habitant_2018
0,Afghanistan,10467.902090,1377.139832,2,34.1,70.3,0,2,6.055574e+08,5.639838e+08
1,Afrique du Sud,55805.974328,3412.061790,202,16.1,46.5,1,202,6.153459e+09,6.412963e+09
2,Albanie,35070.201990,184.077685,3,-4.0,60.3,2,3,4.445132e+09,5.131909e+09
3,Algérie,35847.201990,1558.082864,4,21.6,59.1,3,4,4.051244e+09,4.136290e+09
4,Allemagne,94154.023284,3910.072148,79,2.5,51.8,4,79,4.464274e+10,4.799347e+10
...,...,...,...,...,...,...,...,...,...,...
169,Émirats arabes unis,45593.104080,561.077685,225,35.8,52.2,199,225,4.018034e+10,4.340764e+10
170,Équateur,26955.714826,115.051790,58,17.5,57.1,200,58,6.273489e+09,6.427998e+09
171,États-Unis d'Amérique,201331.795622,4139.062148,231,7.8,35.7,202,231,6.015982e+10,6.298139e+10
172,Éthiopie,20578.170149,1917.093222,238,31.7,67.5,203,238,7.315802e+08,7.466642e+08


In [48]:
data_full_1=data_full.copy()

## 5. Export poulet value

In [49]:
poulet_export_value.head(3)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,TA,Animaux vivants,202,Afrique du Sud,5922,Exportations - Valeur,1057,Poulets,2017,2017,1000 US$,7992,NaN,Donnée officielle
1,TA,Animaux vivants,202,Afrique du Sud,5922,Exportations - Valeur,1057,Poulets,2018,2018,1000 US$,9671,NaN,Donnée officielle
2,TA,Animaux vivants,79,Allemagne,5922,Exportations - Valeur,1057,Poulets,2017,2017,1000 US$,654322,NaN,Donnée officielle


In [50]:
col_mult = ["Valeur"]
for i in col_mult:
    poulet_export_value[i] *= 1000
poulet_export_value.head(5)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,TA,Animaux vivants,202,Afrique du Sud,5922,Exportations - Valeur,1057,Poulets,2017,2017,1000 US$,7992000,NaN,Donnée officielle
1,TA,Animaux vivants,202,Afrique du Sud,5922,Exportations - Valeur,1057,Poulets,2018,2018,1000 US$,9671000,NaN,Donnée officielle
2,TA,Animaux vivants,79,Allemagne,5922,Exportations - Valeur,1057,Poulets,2017,2017,1000 US$,654322000,NaN,Donnée officielle
3,TA,Animaux vivants,79,Allemagne,5922,Exportations - Valeur,1057,Poulets,2018,2018,1000 US$,680352000,NaN,Donnée officielle
4,TA,Animaux vivants,8,Antigua-et-Barbuda,5922,Exportations - Valeur,1057,Poulets,2017,2017,1000 US$,6000,NaN,Donnée officielle


In [51]:
poulet_export_value = poulet_export_value.pivot_table(index=["Zone", "Code zone"], columns=["Année"], values=["Valeur"])

In [52]:
poulet_export_value = poulet_export_value.reset_index()

In [53]:
poulet_export_value.columns = ["Zone", "Code zone", "2017", "2018"]

In [54]:
poulet_export_value = poulet_export_value.reset_index()

In [55]:
poulet_export_value = poulet_export_value.rename(columns={"2017":"Exportations Valeur 2017 usd", "2018":"Exportations Valeur 2018 usd"})

In [56]:
poulet_export_value.head(3)

,index,Zone,Code zone,Exportations Valeur 2017 usd,Exportations Valeur 2018 usd
0,0,Afrique du Sud,202,7992000.0,9671000.0
1,1,Allemagne,79,654322000.0,680352000.0
2,2,Antigua-et-Barbuda,8,6000.0,2000.0


In [57]:
print(pd.isnull(poulet_export_value).sum())

index                            0
Zone                             0
Code zone                        0
Exportations Valeur 2017 usd     4
Exportations Valeur 2018 usd    12
dtype: int64


In [58]:
poulet_export_value = poulet_export_value.fillna(poulet_export_value.mean())

In [59]:
poulet_export_value.shape

(104, 5)

## Poulet Import value

In [60]:
poluet_import_value.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,TA,Animaux vivants,2,Afghanistan,5622,Importations - Valeur,1057,Poulets,2017,2017,1000 US$,8204,R,Données estimées en utilisant les données des ...
1,TA,Animaux vivants,2,Afghanistan,5622,Importations - Valeur,1057,Poulets,2018,2018,1000 US$,9327,R,Données estimées en utilisant les données des ...
2,TA,Animaux vivants,202,Afrique du Sud,5622,Importations - Valeur,1057,Poulets,2017,2017,1000 US$,7394,NaN,Donnée officielle
3,TA,Animaux vivants,202,Afrique du Sud,5622,Importations - Valeur,1057,Poulets,2018,2018,1000 US$,8671,NaN,Donnée officielle
4,TA,Animaux vivants,3,Albanie,5622,Importations - Valeur,1057,Poulets,2017,2017,1000 US$,3266,NaN,Donnée officielle


In [61]:
col_mult = ["Valeur"]
for i in col_mult:
    poluet_import_value[i] *= 1000
poluet_import_value.head(5)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,TA,Animaux vivants,2,Afghanistan,5622,Importations - Valeur,1057,Poulets,2017,2017,1000 US$,8204000,R,Données estimées en utilisant les données des ...
1,TA,Animaux vivants,2,Afghanistan,5622,Importations - Valeur,1057,Poulets,2018,2018,1000 US$,9327000,R,Données estimées en utilisant les données des ...
2,TA,Animaux vivants,202,Afrique du Sud,5622,Importations - Valeur,1057,Poulets,2017,2017,1000 US$,7394000,NaN,Donnée officielle
3,TA,Animaux vivants,202,Afrique du Sud,5622,Importations - Valeur,1057,Poulets,2018,2018,1000 US$,8671000,NaN,Donnée officielle
4,TA,Animaux vivants,3,Albanie,5622,Importations - Valeur,1057,Poulets,2017,2017,1000 US$,3266000,NaN,Donnée officielle


In [62]:
poluet_import_value = poluet_import_value.pivot_table(index=["Zone", "Code zone"], columns=["Année"], values=["Valeur"]).reset_index()

In [63]:
poluet_import_value.columns = ["Zone", "Code zone", "2017", "2018"]

In [64]:
poluet_import_value = poluet_import_value.reset_index()

In [65]:
poluet_import_value = poluet_import_value.rename(columns={"2017":"Importations Valeur 2017", "2018":"Importations Valeur 2018"})

In [66]:
poluet_import_value.head()

,index,Zone,Code zone,Importations Valeur 2017,Importations Valeur 2018
0,0,Afghanistan,2,8204000.0,9327000.0
1,1,Afrique du Sud,202,7394000.0,8671000.0
2,2,Albanie,3,3266000.0,3829000.0
3,3,Algérie,4,19051000.0,26716000.0
4,4,Allemagne,79,193759000.0,185487000.0


In [67]:
plt_imp_exp = pd.merge(poluet_import_value, poulet_export_value, left_on="Zone", right_on="Zone")

In [68]:
plt_imp_exp.head()

,index_x,Zone,Code zone_x,Importations Valeur 2017,Importations Valeur 2018,index_y,Code zone_y,Exportations Valeur 2017 usd,Exportations Valeur 2018 usd
0,1,Afrique du Sud,202,7394000.0,8671000.0,0,202,7992000.0,9671000.0
1,4,Allemagne,79,193759000.0,185487000.0,1,79,654322000.0,680352000.0
2,6,Antigua-et-Barbuda,8,35000.0,84000.0,2,8,6000.0,2000.0
3,7,Arabie saoudite,194,17495000.0,19146000.0,3,194,12897000.0,7157000.0
4,8,Argentine,9,14940000.0,13863000.0,4,9,1445000.0,29925500.0


In [69]:
plt_imp_exp = plt_imp_exp.drop(["index_x", "Code zone_x", "index_y", "Code zone_y"], axis=1)

In [70]:
plt_imp_exp.head()

,Zone,Importations Valeur 2017,Importations Valeur 2018,Exportations Valeur 2017 usd,Exportations Valeur 2018 usd
0,Afrique du Sud,7394000.0,8671000.0,7992000.0,9671000.0
1,Allemagne,193759000.0,185487000.0,654322000.0,680352000.0
2,Antigua-et-Barbuda,35000.0,84000.0,6000.0,2000.0
3,Arabie saoudite,17495000.0,19146000.0,12897000.0,7157000.0
4,Argentine,14940000.0,13863000.0,1445000.0,29925500.0


In [71]:
plt_imp_exp = plt_imp_exp.fillna(plt_imp_exp.mean())
plt_imp_exp.head(3)

,Zone,Importations Valeur 2017,Importations Valeur 2018,Exportations Valeur 2017 usd,Exportations Valeur 2018 usd
0,Afrique du Sud,7394000.0,8671000.0,7992000.0,9671000.0
1,Allemagne,193759000.0,185487000.0,654322000.0,680352000.0
2,Antigua-et-Barbuda,35000.0,84000.0,6000.0,2000.0


In [72]:
data_full_final = pd.merge(data_full_1, plt_imp_exp, left_on="Zone", right_on="Zone")

In [73]:
data_full_final.head()

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Code zone_x,ratio_evol_pop,ratio_protein_anim,index,Code zone_y,Pib_par_habitant_2017,pib_par_habitant_2018,Importations Valeur 2017,Importations Valeur 2018,Exportations Valeur 2017 usd,Exportations Valeur 2018 usd
0,Afrique du Sud,55805.974328,3412.061790,202,16.1,46.5,1,202,6.153459e+09,6.412963e+09,7394000.0,8671000.0,7992000.0,9671000.0
1,Allemagne,94154.023284,3910.072148,79,2.5,51.8,4,79,4.464274e+10,4.799347e+10,193759000.0,185487000.0,654322000.0,680352000.0
2,Antigua-et-Barbuda,47683.738109,276.093222,8,12.8,51.1,8,8,1.439025e+10,1.562905e+10,35000.0,84000.0,6000.0,2000.0
3,Arabie saoudite,43484.242388,1513.093222,194,30.2,51.8,9,194,2.090539e+10,2.331989e+10,17495000.0,19146000.0,12897000.0,7157000.0
4,Argentine,104493.657313,202.077685,9,10.7,27.7,10,9,1.451729e+10,1.160189e+10,14940000.0,13863000.0,1445000.0,29925500.0


In [74]:
data_full_final = data_full_final.drop(["Code zone_x","index", "Code zone_y"], axis=1)

In [75]:
data_full_final.head()

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),ratio_evol_pop,ratio_protein_anim,Pib_par_habitant_2017,pib_par_habitant_2018,Importations Valeur 2017,Importations Valeur 2018,Exportations Valeur 2017 usd,Exportations Valeur 2018 usd
0,Afrique du Sud,55805.974328,3412.061790,16.1,46.5,6.153459e+09,6.412963e+09,7394000.0,8671000.0,7992000.0,9671000.0
1,Allemagne,94154.023284,3910.072148,2.5,51.8,4.464274e+10,4.799347e+10,193759000.0,185487000.0,654322000.0,680352000.0
2,Antigua-et-Barbuda,47683.738109,276.093222,12.8,51.1,1.439025e+10,1.562905e+10,35000.0,84000.0,6000.0,2000.0
3,Arabie saoudite,43484.242388,1513.093222,30.2,51.8,2.090539e+10,2.331989e+10,17495000.0,19146000.0,12897000.0,7157000.0
4,Argentine,104493.657313,202.077685,10.7,27.7,1.451729e+10,1.160189e+10,14940000.0,13863000.0,1445000.0,29925500.0


In [76]:
print(pd.isnull(data_full_final).sum())

Zone                                                        0
Disponibilité alimentaire (Kcal/personne/jour)              0
Disponibilité de protéines en quantité (g/personne/jour)    0
ratio_evol_pop                                              0
ratio_protein_anim                                          0
Pib_par_habitant_2017                                       0
pib_par_habitant_2018                                       0
Importations Valeur 2017                                    0
Importations Valeur 2018                                    0
Exportations Valeur 2017 usd                                0
Exportations Valeur 2018 usd                                0
dtype: int64


In [77]:
data_full_final.shape

(96, 11)

In [78]:
#data_full_final.to_csv('df_final_etude_de_marche_part_1.csv',index=False)